# Dynamisches Labyrinth

## TODOS

1. CSV einlesen

2. Spielsteine definieren

3. A* anhand lösbarem Beispiel

- checken grenzen
    - wenn laufen nicht möglich, dann schieben
        
- verschieben  
    - Zeile über Spielfigur in möglich Richtung
    
- checken funktion/algo
- Listen führen: Open/Close

- Heuristik:
    - 1 Schritt = +1
    - 1 Verschieben = +1
    - Heuristischer wert = Schritte bis zum Ziel
        -   ABER wenn nicht Öffnung nach oben +2

WAS TUN BEI GLEICHSTAND?
AB WANN VERSCHIEBEN (auch wenn laufen möglich)

- jeden möglichen weg berechnen
- jeden möglichen weg nach verschiebungen